# Logistic Regression training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **autoML** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/503296075867861/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Navigate to the parent notebook [here](#notebook/503296075867851) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _10.3.x-cpu-ml-scala2.12_

In [0]:
import mlflow
import databricks.automl_runtime

# Use MLflow to track experiments
mlflow.set_experiment("/Users/cw441@duke.edu/databricks_automl/HeartDisease_heart_2020_cleaned_csv-2022_03_23-14_35")

target_col = "HeartDisease"

## Load Data

In [0]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("e9a782a368624f4c9a32fa566d5033f2", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
0,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes,No
1,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No,No
2,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No,No
3,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes,No
4,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No,No


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`[]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["DiffWalking", "Stroke", "PhysicalHealth", "AgeCategory", "BMI", "AlcoholDrinking", "MentalHealth", "Sex", "GenHealth", "Smoking", "Asthma", "KidneyDisease", "PhysicalActivity", "Diabetic", "SleepTime", "SkinCancer", "Race"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

In [0]:
transformers = []

### Numerical columns

Missing values for numerical columns are imputed with mean for consistency

In [0]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors="coerce"))),
    ("imputer", SimpleImputer(strategy="mean"))
])

transformers.append(("numerical", numerical_pipeline, ["BMI", "SleepTime", "MentalHealth", "PhysicalHealth"]))

### Categorical columns

#### Low-cardinality categoricals
Convert each low-cardinality categorical column into multiple binary columns through one-hot encoding.
For each input categorical column (string or numeric), the number of output columns is equal to the number of unique values in the input column.

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(handle_unknown="ignore")

transformers.append(("onehot", one_hot_encoder, ["DiffWalking", "Stroke", "AlcoholDrinking", "Sex", "GenHealth", "Asthma", "Smoking", "KidneyDisease", "PhysicalActivity", "Diabetic", "AgeCategory", "SkinCancer", "Race"]))

In [0]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

### Feature standardization
Scale all feature columns to be centered around zero with unit variance.

In [0]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [0]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=647181526, stratify=split_y)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=647181526, stratify=split_y_rem)

## Train classification model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/503296075867861/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
from sklearn.linear_model import LogisticRegression

help(LogisticRegression)

Help on class LogisticRegression in module sklearn.linear_model._logistic:

class LogisticRegression(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the
 |  cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag', 'saga' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', 's

In [0]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display="diagram")

sklr_classifier = LogisticRegression(
  C=0.029289025107093664,
  l1_ratio=0.062314014564556465,
  penalty="elasticnet",
  solver="saga",
  random_state=647181526,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
    ("classifier", sklr_classifier),
])

model

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['DiffWalking', 'Stroke', 'PhysicalHealth',
 'AgeCategory', 'BMI', 'AlcoholDrinking',
 'MentalHealth', 'Sex', 'GenHealth',
 'Smoking', 'Asthma', 'KidneyDisease',
 'PhysicalActivity', 'Diabetic',
 'SleepTime', 'SkinCancer', 'Race'])),
 ('preprocessor',
 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transf...
 OneHotEncoder(handle_unknown='ignore'),
 ['DiffWalking', 'Stroke',
 'AlcoholDrinking', 'Sex',
 'GenHealth', 'Asthma',
 'Smoking', 'KidneyDisease',
 'PhysicalActivity',
 'Diabetic', 'AgeCategory',
 'SkinCancer', 'Race'])])),
 ('standardizer', StandardScaler()),
 ('classifier',
 LogisticRegression(C=0.029289025107093664,
 l1_ratio=0.062314014564556465,
 penalty='elasticnet',
 random_state=647181526, solver='saga'))]) ColumnSelector ColumnSelector(cols=['DiffWalking', 'Stroke', 'PhysicalHealth', 'AgeCategory',
 'BMI', 'AlcoholDrinking', 'MentalHealth', 'Sex',
 'GenHealth', 'Smoking', 'Asthma', 'KidneyDisease',
 'PhysicalActivity', 'Diabetic', 'SleepTime', 'SkinCancer',
 'Race']) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('numerical',
 Pipeline(steps=[('converter',
 FunctionTransformer(func= at 0x7faf379a28b0>)),
 ('imputer', SimpleImputer())]),
 ['BMI', 'SleepTime', 'MentalHealth',
 'PhysicalHealth']),
 ('onehot',
 OneHotEncoder(handle_unknown='ignore'),
 ['DiffWalking', 'Stroke', 'AlcoholDrinking',
 'Sex', 'GenHealth', 'Asthma', 'Smoking',
 'KidneyDisease', 'PhysicalActivity',
 'Diabetic', 'AgeCategory', 'SkinCancer',
 'Race'])]) numerical ['BMI', 'SleepTime', 'MentalHealth', 'PhysicalHealth'] FunctionTransformer FunctionTransformer(func= at 0x7faf379a28b0>) SimpleImputer SimpleImputer() onehot ['DiffWalking', 'Stroke', 'AlcoholDrinking', 'Sex', 'GenHealth', 'Asthma', 'Smoking', 'KidneyDisease', 'PhysicalActivity', 'Diabetic', 'AgeCategory', 'SkinCancer', 'Race'] OneHotEncoder OneHotEncoder(handle_unknown='ignore') remainder passthrough passthrough StandardScaler StandardScaler() LogisticRegression LogisticRegression(C=0.029289025107093664, l1_ratio=0.062314014564556465,
 penalty='elasticnet', random_state=647181526, solver='saga')

In [0]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(run_name="logistic_regression") as mlflow_run:
    model.fit(X_train, y_train)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    sklr_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    sklr_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    sklr_val_metrics = {k.replace("val_", ""): v for k, v in sklr_val_metrics.items()}
    sklr_test_metrics = {k.replace("test_", ""): v for k, v in sklr_test_metrics.items()}
    display(pd.DataFrame([sklr_val_metrics, sklr_test_metrics], index=["validation", "test"]))

,precision_score,recall_score,f1_score,accuracy_score,log_loss,roc_auc_score,score
validation,0.889502,0.915774,0.888951,0.915774,0.227532,0.839667,0.915774
test,0.887495,0.914961,0.888781,0.914961,0.227163,0.841717,0.914961


## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = False

In [0]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, len(X_train.index)))

    # Sample a single example from the validation set to explain. Increase the sample size and rerun for more thorough results.
    example = X_val.sample(n=1)

    # Use Kernel SHAP to explain feature importance on the example from the validation set.
    predict = lambda x: model.predict_proba(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="logit")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example, class_names=model.classes_)

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/9b2a518bb3b74d8db0c252c25acd4368/model
